# Examining ROI for images with tumours

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import ipywidgets as widgets
from IPython.display import display

data_path = '/home/szelesteya/projects/EMBED_Open_Data/'
image_root_path = '/media/szelesteya/F824D4D024D492CC/EMBED-images/'
image_figure_path = image_root_path + 'extract'

calc_findings = pd.read_csv(data_path + 'positive_empirical_png.csv')

In [2]:
def parse_ROI(roi_coords_row):
    roi_coords_array = roi_coords_row[2:-1].split('(')
    roi_rects = []
    
    for i in range(len(roi_coords_array)):
        roi_coords_str = roi_coords_array[i].split(')')[0].replace(" ","").split(',')
        if "" not in roi_coords_str:
            try:
                roi_coords = [eval(j.replace('[', '').replace(']','')) for j in roi_coords_str]
    
                x_min = roi_coords[1]
                y_min = roi_coords[0]
                x_max = roi_coords[3]
                y_max = roi_coords[2]
                
                roi_rects.append([x_min, y_min, x_max, y_max])
            except SyntaxError:
                print(roi_coords_str)

    return roi_rects

In [3]:
def showcase_ROI(calcfinding_row, save):
    image = Image.open(calcfinding_row['png_path'])
    image_array = np.array(image)
        
    path = (image_figure_path + '/' + '.'.join(calcfinding_row['relative_dcm_path'].split('/')))[:-4]
    side = calcfinding_row['side']
    # path = calcfinding_row['resized_path']
    
    only_ROI_arrays = []
    print(calcfinding_row[['side','calc_find','calc_distrib','num_find','asses']])
    
    # Coordinate(s) of any detected ROI on the image, represented as a list of lists. 
    # Sublists contains corner coordinates for ROI in the format ‘ymin, xmin, ymax, xmax’.
    roi_coords_array = parse_ROI(calcfinding_row['ROI_coords'])

    aspect_ratio = image_array.shape[1] / image_array.shape[0]
    fig_all, axis_all = plt.subplots(figsize=(4*aspect_ratio, 4))
    axis_all.imshow(image_array, cmap='gray')
    
    for roi in roi_coords_array:
        x_min = roi[0]
        x_max = roi[2]
        y_min = roi[1]
        y_max = roi[3]
        if side == 'R':
            width = image_array.shape[1]
            x_min = width - roi[2]
            x_max = width - roi[0]
            print(f"{x_min} {x_max}")
        rectangle = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=1, edgecolor='r', facecolor='none')
        axis_all.add_patch(rectangle)
        
        only_ROI_arrays.append(image_array[y_min:y_max, x_min:x_max])

    axis_all.set_aspect('equal', adjustable='datalim')
    plt.axis('off')
    if save:
        plt.savefig(path + '_all.png', bbox_inches='tight', pad_inches=0, dpi=300)
    else:
        plt.imshow(image_array, cmap='gray')

    fig, axs = plt.subplots(1, len(only_ROI_arrays), figsize=(15, 5))
    for i in range(len(roi_coords_array)):
        if len(roi_coords_array) > 1:
            axs[i].axis('off')
            axs[i].imshow(only_ROI_arrays[i], cmap='gray')
        else:
            axs.axis('off')
            axs.imshow(only_ROI_arrays[i], cmap='gray')
        
    if save:
        fig.savefig(path + '_rois.png',  bbox_inches='tight', pad_inches=0, dpi=300)

In [4]:
value = 0

def on_value_change(num):
    global value
    value = num
    showcase_ROI(calc_findings.loc[value,:], False)        
    #print(calc_findings.loc[value,:])

def save_figure(b):
    global value
    with button_output:
        showcase_ROI(calc_findings.loc[value,:], True)

In [5]:
# Create a slider for input
number_slider = widgets.IntSlider(value=1, min=1, max=len(calc_findings), description='Number:')

button_save = widgets.Button(description="Save figures")
button_save.on_click(save_figure)

button_output = widgets.Output()

# Create an interactive output that updates when the slider value changes
slider_output = widgets.interactive_output(on_value_change, {'num': number_slider})

# Display the slider and the output
display(number_slider, slider_output)
display(button_save, button_output)

IntSlider(value=1, description='Number:', max=597, min=1)

Output()

Button(description='Save figures', style=ButtonStyle())

Output()